Pythonの基本操作をまとめて、分かること、分からないことを整理したい

前処理
1.レコードデータ
・csv読み込み
・Excel読み込み
・ヘッダーデータの加工
・データ型の変換

2.対象データを抽出
・Filter
・Select

3.マスタデータを結合
・Join
・Union

4.データを集約

5.データ内容の変換
・CSVにアウトプット


#データの新規作成
df = pd.DataFrame({
        'A' : [1, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 7, 8, 9, 10],
        'B' : [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 6, 7, 7, 8, 8, 8, 8, 8, 8]
    })

#CSV読み込み
data  = pd.read_csv(r'フォルダ\ファイル名')
data  = pd.read_excel(r'フォルダ\ファイル名')

#Select <列column抽出>ラベル名から抽出カラムの指定
data = data[['column1', 'column2']]

#Filter<行row抽出>条件を指定して抽出
data = data[data.column1 == 'hoge']


#欠測値(NaN)の除去
欠測値の一つでも存在する行を除去
df = df.dropna()

#項目を指定
df = df.dropna(subset=['項目1', '項目2'])

dplyr使いのためのpandas 基本編
https://qiita.com/T_Shinomiya/items/d445ce2f3d95160ec944#mutate--eval-assign

＜dplyr→pandas＞
1. select(列名)→filter(['列名'])	
 列選択関数。filterだけど列選択なのでdplyr使いは困惑。[['列名']] でも可

    R:dataframe %>% select(trainline, station, year_1975)
    Python:
    dataframe.filter(['trainline', 'station', 'year_1975'])

        trainline    station    year_1975
    0   横浜線        橋本駅     31954.0  
    1   横浜線        相模原駅   29144.0 
    2   横浜線        矢部駅     6706.0


2. filter(抽出条件式)→query('抽出条件式')	
    行選択関数。dplyrと使い方は一緒。やり方はいろいろあるけど、最初はこれがとっつきやすい
    ※ただqueryの問題点としてNoneや欠損値NaNがある列に対して適用しようとするとエラーになる。
    fillna(0), fillna({'列名': '置き換え文字'}) 等で置き換えしておく必要があるようです。

    Python:
    dataframe.query('station == "町田駅（参考）"')

        trainline    station        year_1975    year_1980    ...
    6   横浜線        町田駅（参考）  54506.0      79366.0      ...
    20  小田急線      町田駅（参考）  154831.0     197393.0     ...


3. summarise()→代用：agg()	集計関数。一番大きな違いがある（後述）→describeでいい気がする
    Python：agg
    dataframe.agg({"year_1975":"mean","station":"count"})

    year_1975    29822.882353
    station         25.000000
    dtype: float64


4. group_by(列名)→groupby(['列名'])	group関数。使い方は一緒

    dataframe.groupby(['trainline']).agg({"year_1975":"mean"})

                year_1975
    trainline   
    中央本線     NaN
    京王線       NaN
    小田急線     67854.600000
    横浜線       26004.333333
    相模線       1948.333333


    dataframe.groupby(['trainline']).agg(new_column =("year_1975","mean")).reset_index()
    ※agg(新しいカラム名 =('集計対象カラム', '関数'))

    trainline  new_column
    中央本線     NaN
    京王線       NaN
    小田急線     67854.600000
    横浜線       26004.333333
    相模線       1948.333333


5. mutate(新列名 = 〇〇)→eval(新列名 = 〇〇)	ほぼ一緒（assign()も可）
    #R 新しい列名"new_column"を作成
    dataframe %>% mutate(new_column = year_1975 * 2) %>% dplyr::select(new_column)

    Python：eval
    #eval
    dataframe.eval('new_column = year_1975 * 2').filter(['new_column'])

        new_column
    0   63908.0
    1   58288.0
    2   13412.0

6. rename(新列名 = 旧列名)	rename(columns={'旧列名' : '新列名')	ほぼ一緒

    Python：rename
    dataframe.rename(columns={'trainline':'rail'})

       railline
    0    横浜線
    ...    ...


データの読み込みと書き出し
http://localhost:8892/notebooks/Documents/JupyterNotebook/%E6%95%B4%E7%90%86.ipynb

pd.read_ファイル形式("ファイルのパス/ファイル名"）
               
import pandas as pd

#CSVファイルを読み込む
#encoding="cp932"がないと文字化けすることがある
data = pd.read_csv("sample.csv", encoding="cp932")

print(type(data))

#出力
#CSVファイルを読み込むとDataFrame型になっている
<class 'pandas.core.frame.DataFrame'>


import pandas as pd

#書き出すDataFrameを定義
index = [2017, 2018, 2019, 2020]
columns = ["部員数", "マネージャー数", "成績",]

data = [
    [30, 2, "１回戦負け"],
    [35, 0, "3回戦負け"],
    [50, 1, "準優勝"],
    [70, 5, "優勝"]
]

df = pd.DataFrame(data=data, index=index, columns=columns)

#CSVファイルを書き出す
#encoding="cp932"がないと文字化けすることがある（今回はしました）
df.to_csv("sample.csv", encoding="cp932")

loc とilocの違い→ラベル名入れるか、座標の数字で示すか
http://localhost:8892/notebooks/Documents/JupyterNotebook/%E6%95%B4%E7%90%86.ipynb

df.loc[行, 列]
df.iloc[開始行番号, 終了列番号]

loc[]の引数にはラベル名（index名、columns名）
#2行目、sepal_length列を取得する
iris.loc[2, "sepal_length"]

#出力結果
4.7

#どちらも絶対座標（行名、列名を指定していない時に割り振られる順番）を指定するとエラーになる
print(df.loc[1, 2])

#出力はエラーメッセージになります

ーーーーーーーーーーー
#４行目、３列目を取得する
iris.iloc[3, 2]

#出力結果
1.5

#ラベル名（index名、columns名)を指定するとエラーになる
iris.iloc[2018, "petal_length"]

#出力はエラーメッセージになります

df.カラム # Series型
df['カラム'] # Series型
df[['カラム']] # DataFrame型